In [1]:
import pandas as pd

weather_data = pd.read_csv("Data/Hannover_weather.csv", sep=";")
weather_data = weather_data.drop(weather_data.columns[4:], axis=1)

## Data types convertation
At first, we need to convert types of columns in pandas frame, because in the next steps visualization won't work right, at least with the numeric Y axis.

In [2]:
weather_data['Timestamp'] = pd.to_datetime(weather_data['Timestamp'])
weather_data['Temperature 1'] = pd.to_numeric(list(map(
    lambda x: x.replace(',', '.'), weather_data['Temperature 1'])))
weather_data['Temperature 2'] = pd.to_numeric(list(map(
    lambda x: x.replace(',', '.'), weather_data['Temperature 2'])))
weather_data['Humidity'] = pd.to_numeric(weather_data['Humidity'])

In [6]:
import matplotlib.pyplot as plt
import matplotlib.dates as md
from datetime import datetime
import os


def dates_for_processing_generator(start_data: pd.Timestamp, end_data: pd.Timestamp):
    # this generator follows logic that is specified in the variant
    iter_year = start_data.year
    iter_month = start_data.month

    while iter_year != end_data.year or iter_month != end_data.month + 1:
        yield datetime(year=iter_year, month=iter_month, day=2)
        iter_month += 1
        if iter_month > 12:
            iter_month = 1
            iter_year += 1


def process_data_to_image(
        data: pd.DataFrame,
        date: pd.Timestamp,
        column: str,
        folder_to_save: str,
        color: str = 'b'):
    fig, ax = plt.subplots()
    ax.xaxis.set_major_formatter(md.DateFormatter('%H:%M'))
    fig.set_size_inches(16, 9)

    next_day = pd.Timestamp(year=date.year, month=date.month, day=date.day + 1)
    data_for_day = data[(date < data['Timestamp'])
                        & (data['Timestamp'] < next_day)]

    ax.plot(
        data_for_day['Timestamp'],
        data_for_day[column],
        color=color,
        label=column)

    image_name = f'{date.year}-{date.month}-{date.day}-{column}'

    plt.title(f'{column} changes for {image_name}')
    plt.savefig(f'{folder_to_save}/{image_name}.png', dpi=300)
    plt.close(fig)


start_data = pd.Timestamp(year=2022, month=1, day=2)
current_date = pd.Timestamp.today()
dates_to_process = list(
    dates_for_processing_generator(start_data, current_date))

plt.xlabel('Time, hour')
plt.ylabel('Temperature, °C')

current_folder = os.getcwd()
folder_for_images = current_folder + '/Images'
if not os.path.exists(folder_for_images):
    os.makedirs(folder_for_images)

for date in dates_to_process:
    folder_for_day_data = folder_for_images + f'/{date.year}-{date.month}-{date.day}'
    if not os.path.exists(folder_for_day_data):
        os.makedirs(folder_for_day_data)
    
    process_data_to_image(
        weather_data, date, 'Temperature 1', folder_for_day_data, 'b')
    process_data_to_image(
        weather_data, date, 'Temperature 2', folder_for_day_data, 'y')
    process_data_to_image(
        weather_data, date, 'Humidity', folder_for_day_data, 'black')

    print(f'finished => {date.date()}')
# without additional closing, the image without any params is displayed
plt.close()


finished => 2022-01-02
finished => 2022-02-02
finished => 2022-03-02
finished => 2022-04-02
finished => 2022-05-02
finished => 2022-06-02
finished => 2022-07-02
finished => 2022-08-02
finished => 2022-09-02
finished => 2022-10-02
finished => 2022-11-02
finished => 2022-12-02
finished => 2023-01-02
finished => 2023-02-02
